In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
import tensorflow

import matplotlib.pyplot as plt
from tensorflow.keras.applications.mobilenet import preprocess_input

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
Train_data = "/content/drive/MyDrive/mask/categories/"
Test_data = "/content/drive/MyDrive/mask/"

train_data_df = pd.DataFrame(columns=['Image_path','Label'])
test_data_df = pd.DataFrame(columns=['Image_path'])

In [ ]:
def load_dataframe(DF,path):
    labels_files = os.listdir(Train_data)
    image_paths = []
    labels = []
    for file in labels_files:
        path_file = path + file +'/'
        list_files = os.listdir(path_file)
        for image_path in list_files:
            image_paths.append(path_file + image_path)
            labels.append(file)
    DF['Image_path'] = image_paths
    DF['Label'] = labels
    return(DF)

In [ ]:
train_data_df = load_dataframe(train_data_df,Train_data)
test_data_df = load_dataframe(test_data_df, Test_data)

train_data_df = shuffle(train_data_df)
test_data_df = shuffle(test_data_df)

encoder = LabelEncoder()
train_data_df['Label'] = encoder.fit_transform(train_data_df['Label'])
#test_data_df['Label'] = encoder.fit_transform(test_data_df['Label'])

In [ ]:
import cv2
def load_image(path):
    img = cv2.imread(path,1)
    # OpenCV loads images with color channels
    # in BGR order. So we need to reverse them
    img = cv2.resize(img,(224,224))
    return img

img = load_image(train_data_df['Image_path'][0])
fig,ax = plt.subplots(1)
ax.imshow(img)
plt.show()

img.shape

# New Section

In [ ]:
x_train = []
for image_path in train_data_df['Image_path'] :
    img = load_image(image_path)
    img = preprocess_input(np.array(img, dtype=np.float32))
    x_train.append(img)
x_train = np.array(x_train)

In [ ]:
x_test = []
for image_path in test_data_df['Image_path'] :
    img = load_image(image_path)
    img = preprocess_input(np.array(img, dtype=np.float32))
    x_test.append(img)
x_test = np.array(x_test)

In [ ]:
y_train = train_data_df['Label'].to_list()
#y_test = test_data_df['Label'].to_list()

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train,num_classes=3)
y_train = y_train.reshape(y_train.shape[0],1,1,y_train.shape[1])
y_test = to_categorical(y_test,num_classes=3)
y_test = y_test.reshape(y_test.shape[0],1,1,y_test.shape[1])

In [ ]:
from tensorflow.keras.layers import Reshape, UpSampling2D, Concatenate, Conv2D,Conv2DTranspose,Dropout,Dense,MaxPool2D
from tensorflow.keras.models import Model

def create_model(trainable=True):
    model = tensorflow.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

    for layer in model.layers:
        layer.trainable = trainable    

    x =  model.output
    x = Conv2D(512, kernel_size=3,padding ='same',activation="relu")(x)
    x = MaxPool2D()(x)
    x = Conv2D(256, kernel_size=3,padding ='same',activation="relu")(x)
    x = MaxPool2D()(x)
    x = Dense(512,activation = 'relu')(x)
    x = Dense(3,activation = 'softmax')(x)
    return(Model(inputs = model.input,outputs = x))

In [ ]:
model = create_model(False)
model.summary()

In [ ]:
import tensorflow
Optimizer = tensorflow.keras.optimizers.Adam(learning_rate = 1e-6,beta_1 = 0.9,decay = 1e-5)

model.compile(optimizer= Optimizer,loss= 'categorical_crossentropy',metrics= 'accuracy')

In [ ]:
EarlyStopping = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss',patience=2,min_delta = 0.01)
# Model Checkpoint
ModelCheckpoint = tensorflow.keras.callbacks.ModelCheckpoint("VGG-{val_loss:.3f}.h5",monitor='val_loss',save_best_only= True,save_weights_only=True)

In [ ]:
model_history = model.fit(x_train,y_train,epochs = 10,batch_size = 1,validation_data = (x_test,y_test),callbacks=[ModelCheckpoint,EarlyStopping])

In [ ]:
def plotModelAccuracy(history, modelname):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['validation_accuracy'])

    plt.title(modelname+' model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train','test'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])

    plt.title(modelname+' model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train','test'], loc='upper left')
    plt.show()

In [ ]:
plotModelAccuracy(model_history, 'VGG16')

In [ ]:
#model.save('C:\\Users\\Kalpesh\\Great lakes\\Object Detection')

In [ ]:
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['validation_accuracy'])
plt.title("VGG16" +' model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
#_model = create_model(True)
#_model.load_weights('pneu_2-0.628.h5')
print(model_history.history.keys())

In [ ]:
#np.array(x_train[20,:,:,:]).reshape(1,224,224,3)

In [ ]:
#pred = _model.predict(np.array(x_train[120,:,:,:]).reshape(1,224,224,3))

In [ ]:
#model_history = self._model.fit(X_train,y_train,batch_size=batch_size, epochs=epochs, callbacks=[checkpoint,reduceLoss], validation_data=(X_Val,y_Val))

In [ ]:
def plotModelAccuracy(self, history, modelname):
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])

  plt.title(modelname+' model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train','test'], loc='upper left')
  plt.show()

  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])

  plt.title(modelname+' model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train','test'], loc='upper left')
  plt.show()

In [ ]:
encoder.classes_[np.argmax(pred)]

In [ ]:
encoder.classes_[np.argmax(y_train[120][0][0])]

In [ ]:
#import os
#import tensorflow as tf
#import pandas as pd
#import numpy as np
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import LabelEncoder
#from sklearn.utils import shuffle
#import cv2
#import matplotlib.pyplot as plt
#from tensorflow.keras.applications.mobilenet import preprocess_input
#from myhdl import *

#def predict(image):
 #   
  #  _model = create_model(True)
   # _model.load_weights('pneu_2-0.628.h5')
    #image = cv2.resize(image, (224,224))
    #pred = _model.predict(np.array(image).reshape(1,224,224,3))
    #lab = encoder.classes_[np.argmax(pred)]
    #return(lab)



In [ ]:
#image = cv2.imread('D:\\Omkar Project\\chest_xray\\train\\normal\\IM-0191-0001.jpeg', 1)
#predict(image)

In [ ]:
#converter = tf.lite.TFLiteConverter.from_keras_model(_model) # path to the SavedModel directory
#converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
#tflite_model = converter.convert()
#with open('mobnet_model1.tflite', 'wb') as f:
#    f.write(tflite_model)